In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

%load_ext autotime
import os
import yaml
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.optimize import minimize
from matplotlib.colors import ListedColormap
from matplotlib.colors import to_hex
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import scipy.ndimage.filters as filters
import scipy.ndimage.morphology as morphology

## Change the file to look at here

In [41]:
cwd = os.getcwd()
path = os.path.join(cwd, 'en_imipramine_4_oo_2d_scan', 'en_imipramine_4_oo_3_1_2_4_n_1_2_4_11_coord.yml')

time: 739 µs


In [42]:
with open(path, 'r') as outfile:
    energy = yaml.load(outfile, Loader=yaml.FullLoader)

time: 499 ms


In [43]:
a = np.zeros((45,45))
for k in energy.keys():
    a[k[0],k[1]] = energy[k]

time: 1.05 ms


In [44]:
label = [str(n*8) for n in range(45)]

time: 4.82 ms


In [45]:
df = pd.DataFrame(a, index=label, columns=label)

time: 1.63 ms


In [46]:
np.nanmin(df.to_numpy())

-457.6884535536219

time: 1.48 ms


In [47]:
def highlight_max(data, color='yellow'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

time: 1.68 ms


In [48]:
def highlight_min(data, color='lightgreen'):
    '''
    highlight the minimum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_min = data == data.min()
        return [attr if v else '' for v in is_min]
    else:  # from .apply(axis=None)
        is_min = data == data.min().min()
        return pd.DataFrame(np.where(is_min, attr, ''),
                            index=data.index, columns=data.columns)

time: 885 µs


In [49]:
cm = ListedColormap(sns.color_palette("coolwarm", 7))
df.style.background_gradient(cmap=cm).highlight_null('white').apply(highlight_max, axis=None).apply(highlight_min, axis=None)

time: 1.86 s


## Substract the minimum

In [50]:
df2 = df - np.nanmin(df.to_numpy())

time: 924 µs


In [51]:
df2.style.background_gradient(cmap=cm).highlight_null('white').apply(highlight_max, axis=None).apply(highlight_min, axis=None)

time: 2 s


In [52]:
df2.style.background_gradient(cmap=cm).highlight_null('white').apply(highlight_max, axis=None).apply(highlight_min, axis=None).set_properties(**{'max-width': '80px', 'font-size': '1pt'})

time: 2.28 s


## Interpolation and find minima

In [61]:
df3 = df.interpolate(method='linear', axis=0, limit_direction='both').interpolate(method='linear', axis=1, limit_direction='both')

time: 12.9 ms


In [62]:
df3.style.background_gradient(cmap=cm).highlight_null('white').apply(highlight_max, axis=None).apply(highlight_min, axis=None)

time: 1.79 s


In [110]:
g = interpolate.RectBivariateSpline(range(45), range(45), df3)

time: 1.13 ms


In [113]:
def detect_local_minima(arr):
    # https://stackoverflow.com/questions/3684484/peak-detection-in-a-2d-array/3689710#3689710
    """
    Takes an array and detects the troughs using the local maximum filter.
    Returns a boolean mask of the troughs (i.e. 1 when
    the pixel's value is the neighborhood maximum, 0 otherwise)
    """
    # define an connected neighborhood
    # http://www.scipy.org/doc/api_docs/SciPy.ndimage.morphology.html#generate_binary_structure
    neighborhood = morphology.generate_binary_structure(len(arr.shape),2)
    # apply the local minimum filter; all locations of minimum value 
    # in their neighborhood are set to 1
    # http://www.scipy.org/doc/api_docs/SciPy.ndimage.filters.html#minimum_filter
    local_min = (filters.minimum_filter(arr, footprint=neighborhood)==arr)
    # local_min is a mask that contains the peaks we are 
    # looking for, but also the background.
    # In order to isolate the peaks we must remove the background from the mask.
    # 
    # we create the mask of the background
    background = (arr==0)
    # 
    # a little technicality: we must erode the background in order to 
    # successfully subtract it from local_min, otherwise a line will 
    # appear along the background border (artifact of the local minimum filter)
    # http://www.scipy.org/doc/api_docs/SciPy.ndimage.morphology.html#binary_erosion
    eroded_background = morphology.binary_erosion(
        background, structure=neighborhood, border_value=1)
    # 
    # we obtain the final mask, containing only peaks, 
    # by removing the background from the local_min mask
    detected_minima = local_min ^ eroded_background
    return np.where(detected_minima)   

time: 4.65 ms


In [114]:
local_minima_locations = detect_local_minima(df3)

time: 105 ms


In [115]:
def run_2d_params(params, func):
    a, b = params
    return func(a, b)

time: 579 µs


In [139]:
x0s = (np.array([x,y]) for x, y in zip(local_minima_locations[0], local_minima_locations[1]))  

time: 522 µs


In [140]:
res_list = list()
for x0 in x0s:
    res = minimize(run_2d_params, x0=x0, args=g, method='Nelder-Mead', tol=1e-12)
    res_list.append(res)

time: 54.5 ms


## Minima  d1, d2, energy. d x 8 = real angle

In [120]:
[(r.x, r.fun) for r in res_list]

[(array([-9.65832435e-05,  9.66096221e+00]), -573.2542250543772),
 (array([-2.48710645e-04,  1.96688056e+01]), -573.2722620207286),
 (array([-7.35638807e-05,  2.62581225e+01]), -573.0945950330182),
 (array([-5.23406488e-05,  4.09838751e+01]), -573.2682737835906),
 (array([ 2.57824696, 18.58211016]), -573.2615241915718),
 (array([ 3.32262243e+00, -1.18947998e-04]), -573.25586100747),
 (array([ 2.16055160e+01, -2.87738529e-04]), -573.2698774607327),
 (array([21.78111293, 41.28160439]), -573.2722688183198),
 (array([36.72915302, 23.62144934]), -573.2548702270789),
 (array([ 3.84739349e+01, -1.14927791e-04]), -573.2549269964854),
 (array([40.57529179,  4.5499332 ]), -573.2624557229909),
 (array([44.6619378 , 40.98176146]), -573.2684620681512)]

time: 3.66 ms
